## Исследование надежности заёмщиков

### Цель исследования

Выяснить, как разные характеристика клиента влияют на факт погашения кредита в срок.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Вопросы, на которые необходимо ответить:

 - Есть ли зависимость между количеством детей и возвратом кредита в срок?
 - Есть ли зависимость между семейным положением и возвратом кредита в срок?
 - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 - Как разные цели кредита влияют на его возврат в срок?
 
### Ход исследования

Входные данные от банка — статистика о платёжеспособности клиентов, файл data.csv.
О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки поищем возможность исправить самые критичные ошибки.

Таким образом, исследование пройдёт в три этапа:

 - Обзор данных.
 - Предобработка данных.
 - Ответы на вопросы.
 
Согласно документации к данным, таблица будет состоять из слудкющих столбцов:
 

 - children — количество детей в семье;
 - days_employed — общий трудовой стаж в днях;
 - dob_years — возраст клиента в годах;
 - education — уровень образования клиента;
 - education_id — идентификатор уровня образования;
 - family_status — семейное положение;
 - family_status_id — идентификатор семейного положения;
 - gender — пол клиента;
 - income_type — тип занятости;
 - debt — имел ли задолженность по возврату кредитов;
 - total_income — ежемесячный доход;
 - purpose — цель получения кредита;

## Обзор данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы можем наблюдать разное кол-во значений в столбцах, скорее всего в данных имеются пропуски, помимо этого столбец "total_income" стоит привести к целоцисленному формату для корректности 

## Педобработка данных

In [4]:
# Посчитаем кол-во пропусков для каждого столбца:

data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Cтолбец с пропущенными значениями — "total_income" — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца "income_type". То же проделаем для столбца "days_employed"

In [5]:
for i in data['income_type'].unique():
    data.loc[(data['income_type']==i) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type']==i), 'total_income'].median()

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [7]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнены корректно

In [8]:
# Изменим тип данных столбца "total_income" на целоцисленный: 

data['total_income'] = data['total_income'].astype(int)

In [9]:
# В столбце "education" названия записаны с заглывных и строчных букв, исправим это:

data['education'] = data['education'].str.lower()

In [10]:
# Проверим наличие дубликатов:

data.duplicated().sum()

71

In [11]:
# Удалим их:

data = data.drop_duplicates()

Разобъем данные на категории: создим столбец "total_income_category" с категориями:

 - 0–30000 — 'E';
 - 30001–50000 — 'D';
 - 50001–200000 — 'C';
 - 200001–1000000 — 'B';
 - 1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'

In [12]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [27]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [28]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,otal_income_category,purpose_cathegory,total_income_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,B
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,C
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,C
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью,B
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем,C
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью,C
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем,B


In [29]:
# Рассмотрим столбоц с целью кредита:

data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Некоторые из схожих целей описаны по-разному, постараемся привести все категории целей к единому виду

In [30]:
def purposes(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [31]:
data['purpose_cathegory'] = data['purpose'].apply(purposes)

In [32]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,otal_income_category,purpose_cathegory,total_income_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,B
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,C
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,C
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,C


## Ответы на вопросы 

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [33]:
# вывидем общее число кредитов для семей с разным кол-вом детей и сумму непогашенных кредитов: 
children_table = data.groupby('children').agg({'debt' : ['count', 'sum']})
children_table

debt      
          count   sum
children             
-1           47     1
 0        14091  1063
 1         4808   444
 2         2052   194
 3          330    27
 4           41     4
 5            9     0
 20          76     8

In [34]:
#Удалим аномальные значения:
children_table = children_table.drop(index = [-1, 20])
children_table

debt      
          count   sum
children             
0         14091  1063
1          4808   444
2          2052   194
3           330    27
4            41     4
5             9     0

In [35]:
#Постичаем процент непогфшенных кредитов офильтрованный по кол-ву детей в семье и отсортируем его в порядке убывания
children_table['percent_debt'] = children_table['debt']['sum'] / children_table['debt']['count'] * 100
children_table.sort_values(by='percent_debt', ascending=False)

debt       percent_debt
          count   sum             
children                          
4            41     4     9.756098
2          2052   194     9.454191
1          4808   444     9.234609
3           330    27     8.181818
0         14091  1063     7.543822
5             9     0     0.000000

**Вывод: Проанализировав данные было выявлено, процент непогашенных кретитов увеличивается в зависимости от кол-ва детей в семье. 3 и 5 детей выбиваются из общей последовательности, это можно связать с тем, что данные с тремя детьми являются погрешностью в общей последовательности, а данных по пяти детям слишком мало для объективной картины, но можно допустить, что описанная зависимоть сохранится.**

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
#Общее число и сумма кредитов скруппированные по семейному статусу:
family_table  = data.groupby('family_status').agg({'debt' : ['count', 'sum']})

#процент непогошенных кредитов:
family_table['percent_debt'] = family_table['debt']['sum'] / family_table['debt']['count'] * 100 

# таблица с семейным положением, отсортированная в порядке убывания непогашенных кредитов
family_table.sort_values(by='percent_debt', ascending=False) 

debt      percent_debt
                       count  sum             
family_status                                 
Не женат / не замужем   2810  274     9.750890
гражданский брак        4151  388     9.347145
женат / замужем        12339  931     7.545182
в разводе               1195   85     7.112971
вдовец / вдова           959   63     6.569343

**Вывод: из представленых данных можно сделать вывод, что наибольший процент невыплаченных кредитов официально не женатых/не замужних клиентов. Наменьший риск невозврата у клиентов, находящихся в разводе и вдов/вдовцов** 

### Естьли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
#Общее число и сумма кредитов скруппированные по уровню дохода:
income_table  = data.groupby('total_income_category').agg({'debt' : ['count', 'sum']})

# Добавим столбец с расшифровкой к категории дохода:
income_table['income_range'] = ['> 500001', '200001 - 500000', '50001 - 200000', '30001 - 50000', ' < 30000'] # Добавим столбец с расшифровкой к категории дохода

#процент непогошенных кредитов:
income_table['percent_debt'] = income_table['debt']['sum'] / income_table['debt']['count'] * 100 

#Таблица, отсортированная по уровню дохода, с процентами невозвращенных кредитов 
income_table.sort_values(by='total_income_category', ascending=False) 

debt           income_range percent_debt
                       count   sum                              
total_income_category                                           
E                         22     2          < 30000     9.090909
D                        350    21    30001 - 50000     6.000000
C                      16015  1360   50001 - 200000     8.492039
B                       5042   356  200001 - 500000     7.060690
A                         25     2         > 500001     8.000000

**Вывод: Из представленных данных, взаимосвязи с ростом уровня дохода и возвратом кредита не обнаружено, однако наибольшее число задолженностей у клиентов с доходом меньше 30000р.**  

### Как разные цели кредита влияют на возврат кредита в срок?

In [44]:
#Общее число и сумма кредитов скруппированные по цели получения:
purpose_table  = data.groupby('purpose_cathegory').agg({'debt' : ['count', 'sum']}) 

#процент непогошенных кредитов:
purpose_table['percent_debt'] = purpose_table['debt']['sum'] / purpose_table['debt']['count'] * 100 

# таблица с целью получения кредита отсортированная в порядке убывания непогашенных кредитов:
purpose_table.sort_values(by='percent_debt', ascending=False)  

debt      percent_debt
                          count  sum             
purpose_cathegory                                
операции с автомобилем     4306  403     9.359034
получение образования      4013  370     9.220035
проведение свадьбы         2324  186     8.003442
операции с недвижимостью  10811  782     7.233373

**Вывод: В зависмости от цели получения крелита процент невозврата меняется. Наибольший риск связан с оперциями с автомобилем, наименьший - с недвижимостью.**

### Общий вывод: 

**Проанализировав представленные данные можно сделать вывод, что доля невозврата кредита зависит от таких факторов, как: кол-во детей в семье, семейного положения, цели получения кредита и частично от уровня дохода клиента банка. Наибольший риск невозврата кредита присущ следующим категориям: "семьи с 4 и более детьми"; "клиент официально не состоит и не состоял в браке"; " ежемесячный доход менее 30000р"; "Опереции с автомобилем"**  